In [1]:
import pandas as pd
import numpy as np
from os import path
from CSVUtils import *
import ta
import matplotlib.pyplot as plt
import seaborn as sn
import calendar
from pprint import pprint

In [2]:
DIR = "./from github/Stock-Trading-Environment/data"
nameList = ["^BVSP", "^TWII", "^IXIC"]
df_list = []
startDate = pd.to_datetime("2015-01-01")
endDate = pd.to_datetime("2019-12-31")

for name in nameList:
    df = csv2df(csv_path=DIR, csv_name=name+".csv", source="yahoo")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna()
    df_list.append(df)

In [3]:
DIR = "./from github/Stock-Trading-Environment/data"
norm_nameList = ["^BVSP_new", "^TWII_new", "^IXIC"]
sources = ['done', 'done', 'yahoo']
norm_df_list = []
startDate = pd.to_datetime("2015-01-01")
endDate = pd.to_datetime("2019-12-31")

for i, name in enumerate(norm_nameList):
    df = csv2df(csv_path=DIR, csv_name=name+".csv", source=sources[i])
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna()
    norm_df_list.append(df)

In [6]:
intersect_dates = df_list[0]['Date']
for df in df_list[1:]+norm_df_list:
    df.dropna(inplace = True)
    intersect_dates = np.intersect1d(intersect_dates, df['Date'])
# Remove all NAN in the df

for i, df in enumerate(df_list):
    df.dropna(inplace = True)
    df = df[df['Date'].isin(intersect_dates)].reset_index(drop=True)
    df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate)]
    df_list[i] = df.reset_index(drop=True)

for i, df in enumerate(norm_df_list):
    df.dropna(inplace = True)
    df = df[df['Date'].isin(intersect_dates)].reset_index(drop=True)
    df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate)]
    norm_df_list[i] = df.reset_index(drop=True)

In [7]:
norm_df_list[0]

,Date,Open,High,Low,Price,Vol,Change
0,2015-01-05,17925.1840,17925.1840,17464.0480,17557.5315,3866100.0,-0.024996
1,2015-01-06,17600.2968,17801.7944,17533.9952,17779.5704,4559300.0,0.012646
2,2015-01-07,17891.8362,18591.0214,17891.8362,18434.8601,4408800.0,0.036856
3,2015-01-08,18573.3565,18873.0055,18405.8835,18753.5965,3621900.0,0.017290
4,2015-01-09,18992.8910,18992.8910,18440.0802,18568.9680,2999200.0,-0.009845
...,...,...,...,...,...,...,...
1132,2019-12-20,28057.9121,28067.1727,27909.9862,28054.9877,6887400.0,-0.008226
1133,2019-12-23,28192.6431,28374.8487,28154.6836,28374.8487,3995200.0,0.011401
1134,2019-12-26,28560.4760,28894.7300,28513.3945,28890.5395,3788900.0,0.018174
1135,2019-12-27,28973.0760,29120.9016,28673.9640,28807.2048,3905900.0,-0.002884


In [8]:
df_list[0]

,Date,Open,High,Low,Price,Vol,Change
0,2015-01-05,48512.0,48512.0,47264.0,47517.0,3866100.0,-0.020510
1,2015-01-06,47517.0,48061.0,47338.0,48001.0,4559300.0,0.010186
2,2015-01-07,48006.0,49882.0,48006.0,49463.0,4408800.0,0.030458
3,2015-01-08,49463.0,50261.0,49017.0,49943.0,3621900.0,0.009704
4,2015-01-09,49955.0,49955.0,48501.0,48840.0,2999200.0,-0.022085
...,...,...,...,...,...,...,...
1132,2019-12-20,115133.0,115171.0,114526.0,115121.0,6887400.0,-0.000087
1133,2019-12-23,115119.0,115863.0,114964.0,115863.0,3995200.0,0.006445
1134,2019-12-26,115864.0,117220.0,115673.0,117203.0,3788900.0,0.011565
1135,2019-12-27,117205.0,117803.0,115995.0,116534.0,3905900.0,-0.005708


In [13]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [51]:
annual_rebalance_date = ["12-31"]
semiAnnual_rebalance_date = ["06-30", "12-31"]
quarter_rebalance_date = ["03-31", "06-30", "09-30", "12-31"]
monthly_rebalance_date = ["01-31", "02-28", "03-31", "04-30", "05-31", "06-30", "07-31", "08-31", "09-30", "10-31", "11-30", "12-31"]

In [72]:
init_net_worth = 300000
avg_net_worth = init_net_worth / (len(df_list))
asset_list = norm_nameList
init_price_arr = np.array([df.Price[0] for df in norm_df_list])
init_num_arr = np.array([avg_net_worth / price for price in init_price_arr])

# For testing only, will be overwritten
status_list = []
for i, asset in enumerate(asset_list):
    status = {
        'Asset': asset_list[i],
        'Number': init_num_arr[i],
        'Price': init_price_arr[i],
        'Net Value': avg_net_worth,
        'Delta Number': init_num_arr[i],
    }
    status_list.append(status)
    
COMMISSION_RATE = 0.00125
year = 2016
balance_date = quarter_rebalance_date

prev_num_arr = init_num_arr
result_list = []
net_value_list = []

result_list.append({
    'Date': norm_df_list[0]['Date'][0],
    'Record': status_list
})

result_list

[{'Date': Timestamp('2015-01-05 00:00:00'),
  'Record': [{'Asset': '^BVSP_new',
    'Number': 5.695561474572892,
    'Price': 17557.5315,
    'Net Value': 100000.0,
    'Delta Number': 5.695561474572892},
   {'Asset': '^TWII_new',
    'Number': 345.15567271965676,
    'Price': 289.7243415182756,
    'Net Value': 100000.0,
    'Delta Number': 345.15567271965676},
   {'Asset': '^IXIC',
    'Number': 21.49349795550752,
    'Price': 4652.569824,
    'Net Value': 100000.0,
    'Delta Number': 21.49349795550752}]}]

In [73]:
for date_str in balance_date:
    # 1) Determine the rebalance date
    rebalance_date = pd.to_datetime(str(year)+'-'+date_str)
    
    # 2) Getting the price of the nearest date after the rebalance date
    price_list = []
    nearest_date_after = nearest(norm_df_list[0]['Date'][norm_df_list[0]['Date']>=rebalance_date], rebalance_date)
    
    for df in norm_df_list:
        price_list.append(df.Price[df.Date==nearest_date_after].values[0])

    # 3) Calculate the buy and sell number
    price_arr = np.array(price_list)
    current_net_worth = sum(prev_num_arr*price_arr)
    current_nominal_num_arr = (current_net_worth/len(asset_list))/price_arr
    
    delta_num_arr = current_nominal_num_arr - prev_num_arr
    
    sell_num_arr = delta_num_arr*(delta_num_arr<0)*-1
    buy_num_arr = delta_num_arr*(delta_num_arr>0)
    
    sell_cash = np.sum(sell_num_arr*price_arr)*(1-COMMISSION_RATE)

    buy_budget_proposed = buy_num_arr*price_arr
    buy_ratio = buy_budget_proposed/np.sum(buy_budget_proposed)
    buy_budget_actual = buy_ratio*sell_cash
    buy_num_arr_actual = buy_budget_actual / (price_arr*(1+COMMISSION_RATE))
    
    # 4. Update the inventory
    current_num_arr = prev_num_arr-sell_num_arr+buy_num_arr_actual
    
    status_list = []
    for i, asset in enumerate(asset_list):
        status = {
            'Asset': asset_list[i],
            'Number': current_num_arr[i],
            'Price': price_arr[i],
            'Net Value': current_num_arr[i]*price_arr[i],
            'Delta Number': current_num_arr[i]-prev_num_arr[i],
        }
        status_list.append(status)

    prev_num_arr = current_num_arr

    result_list.append({
        'Date': nearest_date_after,
        'Record': status_list
    })

    net_value_list.append({
        'Date': nearest_date_after,
        'Net Value': current_net_worth
    })


2016-03-31 00:00:00
[  6.64326602 340.95513628  19.00998508] [92542.73183724 92575.77768156 92575.77768156]
2016-06-30 00:00:00
[  6.03472659 359.67764045  19.97995035] [96768.06233992 96755.4554148  96756.30458861]
2016-09-30 00:00:00
[  5.94778783 363.82772446  20.04357099] [106472.29503047 106469.25506584 106471.44909093]
2016-12-31 00:00:00
[  5.73697019 378.38164211  20.0073786 ] [108621.66056119 108611.20354698 108621.66056119]


In [44]:
result_list

[{'Date': Timestamp('2015-01-05 00:00:00'),
  'Record': [{'Asset': '^BVSP_new',
    'Number': 5.695561474572892,
    'Price': 17557.5315,
    'Net Value': 100000.0,
    'Delta Number': 5.695561474572892},
   {'Asset': '^TWII_new',
    'Number': 345.15567271965676,
    'Price': 289.7243415182756,
    'Net Value': 100000.0,
    'Delta Number': 345.15567271965676},
   {'Asset': '^IXIC',
    'Number': 21.49349795550752,
    'Price': 4652.569824,
    'Net Value': 100000.0,
    'Delta Number': 21.49349795550752}]},
 {'Date': Timestamp('2016-03-31 00:00:00'),
  'Record': [{'Asset': '^BVSP_new',
    'Number': 5.695561474572892,
    'Price': 13930.3065,
    'Net Value': 79340.91703039235,
    'Delta Number': 0.0},
   {'Asset': '^TWII_new',
    'Number': 345.1451844888447,
    'Price': 271.5189414413016,
    'Net Value': 93713.45513597385,
    'Delta Number': -0.010488230812086385},
   {'Asset': '^IXIC',
    'Number': 21.487296924595455,
    'Price': 4869.850098,
    'Net Value': 104639.91503399

In [ ]:
start_year = startDate.year
final_year = endDate.year
rebalance_freq = ['annual_rebalance', 
                  'semiAnnual_rebalance', 
                  'quarter_rebalance', 
                  'monthly_rebalance']

final_result = []
for j, balance_date in enumerate(
    [annual_rebalance_date, semiAnnual_rebalance_date, quarter_rebalance_date, monthly_rebalance_date]):
    result_list = []
    
    # 1. Init the final result and the holding number
    result_list.append({
        'Date': df_list[0]['Date'][0],
        'Record': status_list
    })
    
    prev_num_list = init_num_list
    net_value_list = []
    
    # 2. Get rebalance data
    for year in range(start_year, final_year):
        for date_str in balance_date:
            # 1) Determine the rebalance date
            rebalance_date = pd.to_datetime(str(year)+'-'+date_str)

            # 2) Getting the price of the nearest date after the rebalance date
            price_list = []
            nearest_date_after = nearest(norm_df_list[0]['Date'][norm_df_list[0]['Date']>=rebalance_date], rebalance_date)
            for df in norm_df_list:
                price_list.append(df.Price[df.Date==nearest_date_after].values[0])

            # 3) Calculate the current value by nominal price & actual price
            price_arr = np.array(price_list)
            current_net_worth = sum(np.array(prev_num_list)*price_arr)
            current_nominal_num_arr = (current_net_worth/len(asset_list))/price_arr
            
            delta_num_arr = current_nominal_num_arr - np.array(prev_num_list)
            
            sell_num_arr = delta_num_arr*(delta_num_arr<0)*-1
            buy_num_arr = delta_num_arr*(delta_num_arr>0)
            
            sell_cash = np.sum(sell_arr*price_arr)
            
            buy_budget = sell_arr*price_arr
            
            status_list = []
            for i, asset in enumerate(asset_list):
                status = {
                    'Asset': asset_list[i],
                    'Number': current_num_list[i],
                    'Price': price_list[i],
                    'Net Value': current_num_list[i]*price_list[i],
                    'Delta Number': current_num_list[i]-prev_num_list[i],
                }
                status_list.append(status)

            prev_num_list = current_num_list

            result_list.append({
                'Date': nearest_date_after,
                'Record': status_list
            })

            net_value_list.append({
                'Date': nearest_date_after,
                'Net Value': current_net_worth
            })

    final_result.append(result_list)
    print(rebalance_freq[j])
print("Done")


In [ ]:
date = final_result[0][0]['Date']
date

In [ ]:
# for bal_result in final_result:
rebal_result = pd.DataFrame(final_result[0])
rebal_date_list = [tmp['Date'] for tmp in rebal_result]
start_date = rebal_date_list[0]
end_date = rebal_date_list[-1]
current_date = start_date
plot_list = []

while current_date <= end_date:
    if current_date in rebal_date_list:
        asset_record_list = rebal_result[rebal_result['Date']==date]['Record'][0]
        inv_net_worth = [tmp['Net Value'] for tmp in asset_record_list]
        inv_number = [tmp['Number'] for tmp in asset_record_list]
        record = {
            'date': record['Date'],
            'inv_number':inv_number
            'inv_net_worth': inv_net_worth,
            'total_net_worth': np.sum(inv)
        }
    else:
        last_record = plot_list[-1]
        if (date == df_list[0]['Date']).any():
            inv_number = last_record['inv_number']
            price = 
    plot_list.append(record)
    current_date += pd.Timedelta(days=1)

In [ ]:
rebal_result = pd.DataFrame(final_result[0])
rebal_result[rebal_result['Date']==date]['Record'][0]

In [ ]:
norm_df_list[0][norm_df_list[0]['Date']==date]

In [ ]:
date

In [ ]:
norm_df_list[0]

In [ ]:
plot_list = []
for rebal_result in final_result:
    tmp_list = []
    for record in rebal_result:
        inv = [tmp['Net Value'] for tmp in record['Record']]
        tmp_list.append({
            "date": record['Date'],
            'inventory': inv,
            'net_worth': np.sum(inv)
        })
    plot_list.append(tmp_list)

In [ ]:
plot_list[0]

In [ ]:
def plot_comparison(record, title, currency = False):
    import matplotlib.pyplot as plt
    import numpy as np

    labels = ["high-^BVSP", "mid-^TWII", "low-^IXIC"]
    x = [tmp['date'] for tmp in record]
    y_dict = {}
    plt.rcParams['figure.facecolor'] = 'white'
    fig, axs = plt.subplots(2,figsize=(40,20))
    plt.suptitle(title, fontsize=16)

    width = 0.9
    for i in range(2,-1,-1): # Inverse: Low-Mid-High
        y = np.array([tmp['inventory'][i] for tmp in record])
        if i == 2:
            axs[0].bar(x, y, width)
            axs[0].plot(x,y, label=labels[i])
            y_dict[i] = y
        else:
            axs[0].bar(x, y, width, bottom=y_dict[i+1])
            axs[0].plot(x,y+y_dict[i+1],label =labels[i])
            y_dict[i] = (y+y_dict[i+1])

    axs[0].axhline(y=300000, color = "grey", linestyle='--')
    axs[0].set_ylim((0, 800000))
    axs[0].legend()
    axs[0].set_title('Portfolio Weights')
    axs[0].set_xlabel('Date')
    axs[0].set_ylabel('Market Value (US$)')




    DIR = "./from github/Stock-Trading-Environment/data"
    if currency:
        file_names = ["^BVSP_new", "^TWII_new", "^IXIC"]
        source_list = ["done", "done", "yahoo"]
    else:
        file_names = ["^BVSP", "^TWII", "^IXIC"]
        source_list = ["yahoo", "yahoo", "yahoo"]
        
    labels = ["high-", "mid-", "low-"]

    # axs[1].figure(figsize = (40,15))
    for i in range(2,-1,-1): # Inverse: Low-Mid-High
        df = csv2df(DIR, file_names[i]+".csv",source = source_list[i])
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[(df['Date']>=pd.to_datetime("2015-01-01"))&(df['Date']<=pd.to_datetime("2019-12-31"))].reset_index(drop=True)

        j = 0
        init_price = df['Price'][j]
        while np.isnan(init_price):
            j+=1
            init_price = df['Price'][j]
        y = np.log(df['Price'][j:] / init_price)
        x = df['Date'][j:]
        axs[1].plot(x,y,label = labels[i]+file_names[i])

    axs[1].plot([tmp['date'] for tmp in record], np.log([tmp['net_worth'] for tmp in record]/record[0]['net_worth']), label = "RL Portfolio")
    axs[1].axhline(y=0, color = "grey", linestyle='--')
    axs[1].set_ylim((-1,1))
    axs[1].legend()
    axs[1].set_title('Log Market Price')
    axs[1].set_xlabel('Date')
    axs[1].set_ylabel('log(Market Price)')
    plt.show()


In [ ]:
plot_comparison(plot_list[0], "Model Behavior Analysis-100k-7(8)",currency=True)

In [ ]:
plot_comparison(plot_list[1], "Model Behavior Analysis-100k-7(8)",currency=True)

In [ ]:
plot_comparison(plot_list[2], "Model Behavior Analysis-100k-7(8)",currency=True)

In [ ]:
plot_comparison(plot_list[3], "Model Behavior Analysis-100k-7(8)",currency=True)